In [64]:
from fractions import Fraction
from typing import Tuple, Union
import numpy as np
import pandas as pd

pd.set_option('display.precision', 7)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [65]:
def input_matrix(filename, convert_fractions=False):
    """
    Reads a matrix from a text file and returns it as a NumPy array.
    Supports fractional entries if present.
    """
    matrix = []

    with open(filename, 'r') as f:
        for line in f:
            tokens = line.strip().split()
            if not tokens:
                continue

            row = []
            for token in tokens:
                if '/' in token:
                    # convert fractions if any includes fraction sign
                    val = Fraction(token)
                    row.append(float(val) if convert_fractions else val)
                else:
                    # parse as float directly
                    row.append(float(token))

            matrix.append(row)
            
    dtype = float if convert_fractions else object
    return np.array(matrix, dtype=dtype)

In [66]:
def output_matrix(X: np.ndarray, precision: int = 12):
    """
    Prints a NumPy array (vector or matrix) in a clean tabular format using pandas.
    
    Parameters:
    - X: np.ndarray, 1D or 2D array.
    - precision: number of decimal places to round floats to.
    """
    # Wrap 1D arrays into a 2D DataFrame for consistent display
    if X.ndim == 1:
        df = pd.DataFrame(X, columns=["value"])
    elif X.ndim == 2:
        df = pd.DataFrame(X)
    else:
        raise ValueError("Only 1D or 2D arrays are supported.")
    
    # Round floats
    # df = df.round(precision)
    # Print without index/header for cleaner look
    print(df.to_string(index=False, header=False))

# Lặp Jacobi

In [67]:
def check_dominance(A: np.ndarray) -> int:
    """
    Returns:
      1 if A is row-dominant only,
      2 if A is column-dominant only,
      3 if both,
      0 if neither.
    """
    n = A.shape[0]
    row_dom = all(abs(A[i,i]) > np.sum(np.abs(A[i,:])) - abs(A[i,i]) for i in range(n))
    col_dom = all(abs(A[j,j]) > np.sum(np.abs(A[:,j])) - abs(A[j,j]) for j in range(n))
    if row_dom and col_dom:
        return 3
    if row_dom:
        return 1
    if col_dom:
        return 2
    return 0

## 1. Ma trận chéo trội Hàng

### Thuật toán

In [68]:
def convert_to_iteration(A: np.ndarray, B: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Converts the system Ax = B into the iterative form x_new = C x + D.
    """

    # Step 1: Build diagonal matrix T where T[i,i] = 1 / A[i,i]
    T = np.diag(1.0 / np.diag(A))
    
    # Step 2: Compute C = I - T * A
    n = A.shape[0]
    C = np.eye(n) - T.dot(A)
    
    # Step 3: Compute D = T * B
    D = T.dot(B)
    
    return C, D

In [69]:
def fixed_point_matrix_iteration(
    A: np.ndarray,
    C: np.ndarray,
    D: np.ndarray,
    x0: Union[np.ndarray, list],
    domiType: int,
    eps: float,
    eta: float
) -> pd.DataFrame:
    """
    Performs fixed-point iteration x_{k+1}=C x_k + D and returns history with errors.
    Convergence based on:
      - exact error if eps is provided (||x_new - x_old|| <= eps*(1-q)/(α*q))
      - relative error if eta is provided (||x_new-x_old||/||x_new|| <= eta*(1-q)/(α*q))

    Parameters:
    - C, D: iteration matrix and constant
    - x0: initial guess
    - domiType: 1 or 3 (row), 2 (column)
    - eps: absolute tolerance (use exact error)
    - eta: relative tolerance (use ||Δx||/||x_new||)
    """
    # Validate tolerance inputs
    if (eps is None) == (eta is None):
        raise ValueError("Specify exactly one of eps (exact) or eta (relative)")

    # Set norms and α, q based on dominance type
    if domiType in (1, 3):
        vec_norm = lambda x: np.max(np.abs(x))
        q = np.max(np.sum(np.abs(C), axis=1))
        alpha = 1.0
    else:
        raise ValueError("domiType must be 1 or 3 for row-dominant iteration")

    # Convergence threshold
    tol = (eps if eps is not None else eta) * (1 - q) / (alpha * q)
    print(f"Shrinking value: {q:.12f}, Additional value: {alpha:.12f}, Threshold: {tol:.12f}")

    # Initialize
    x_old = np.array(x0, dtype=float).flatten()
    history = [x_old.copy()]
    errors = [np.nan]

    # Iteration loop
    while True:
        x_new = C.dot(x_old) + D
        if eps is not None:
            err = vec_norm(x_new - x_old)
        else:
            err = vec_norm(x_new - x_old) / vec_norm(x_new)

        # Prepare DataFrame for display
        history.append(x_new.copy())
        errors.append(err)
        x_old = x_new
        if err <= tol:
            break
        
    # Compile DataFrame
    n = len(x_new)
    df = pd.DataFrame(history, columns=[f"x{i+1}" for i in range(n)])
    df["error"] = errors
    df.index.name = "Iteration"
    return df

### Kết quả

In [70]:
#Original matrix Ax=B
A = input_matrix('JCB_input_A1.txt', convert_fractions=False)
B = input_matrix('JCB_input_B1.txt', convert_fractions=False).flatten() #remove flatten if B is multi-column matrix

print("\nMatrix A:"); output_matrix(A)
print("\nCheck dominace of A:", check_dominance(A));
print("\nMatrix B:"); output_matrix(B)


Matrix A:
 14.4   0.2   0.6  -0.1   0.9   0.4   0.9
  0.4  16.3  -0.4  -0.7  -0.5   0.1  -0.6
  0.7   0.9  18.6  -0.3   0.6  -0.2   0.4
 -0.3  -1.0  -0.1  19.7  -0.7   0.3  -0.6
  0.6  -0.1  -1.0  -0.6  20.6   0.3  -0.8
  0.4  -0.2  -0.7   0.5  -0.9  22.9   0.2
 -1.0  -0.1   0.5  -0.5   0.2   0.3  23.9

Check dominace of A: 3

Matrix B:
-10.0
 -3.0
 -7.0
  6.0
 -4.0
  1.0
 -7.0


In [71]:
#Convert to recursion form x_new = Cx+D
C, D = convert_to_iteration(A, B)

print("\nMatrix C:"); output_matrix(C)
print("\nMatrix D:"); output_matrix(D)


Matrix C:
       0.0 -0.0138889 -0.0416667  0.0069444    -0.0625 -0.0277778    -0.0625
-0.0245399        0.0  0.0245399  0.0429448  0.0306748  -0.006135  0.0368098
-0.0376344 -0.0483871        0.0   0.016129 -0.0322581  0.0107527 -0.0215054
 0.0152284  0.0507614  0.0050761        0.0   0.035533 -0.0152284  0.0304569
-0.0291262  0.0048544  0.0485437  0.0291262        0.0 -0.0145631   0.038835
-0.0174672  0.0087336  0.0305677 -0.0218341  0.0393013        0.0 -0.0087336
  0.041841  0.0041841 -0.0209205  0.0209205 -0.0083682 -0.0125523        0.0

Matrix D:
-0.6944444
-0.1840491
-0.3763441
 0.3045685
-0.1941748
 0.0436681
 -0.292887


In [73]:
#Calculate the result
x0 = np.array([0,0,0,0,0,0,0]) #initial value
domiType = check_dominance(A)
eps = None
eta = 1e-6

df_history = fixed_point_matrix_iteration(A, C, D, x0, domiType, eps, eta)
print(df_history)


Shrinking value: 0.215277777778, Additional value: 1.000000000000, Threshold: 0.000003645161
                  x1         x2         x3         x4         x5         x6         x7      error
Iteration                                                                                        
0          0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000        NaN
1         -0.6944444 -0.1840491 -0.3763441  0.3045685 -0.1941748  0.0436681 -0.2928870  1.0000000
2         -0.6448638 -0.1801686 -0.3233592  0.2662552 -0.1962501  0.0309634 -0.3073916  0.0821645
3         -0.6460023 -0.1822500 -0.3257886  0.2671542 -0.1965974  0.0326326 -0.3070340  0.0037607
4         -0.6459129 -0.1822509 -0.3256091  0.2669920 -0.1966765  0.0325236 -0.3070388  0.0002779
5         -0.6459133 -0.1822575 -0.3256135  0.2669929 -0.1966738  0.0325280 -0.3070401  0.0000104
6         -0.6459132 -0.1822576 -0.3256132  0.2669925 -0.1966741  0.0325279 -0.3070401  0.0000006


In [ ]:
solution_series = df_history.filter(regex=r'^x\d+$').iloc[-1]
print("Approximate solution:"),
print(solution_series.to_string())

Approximate solution:
x1     1.5200001
x2    -1.5400002
x3     2.5000001
x4    -2.1400004
x5     1.4999999
x6     0.9999998
x7     0.9999995
x8     2.0000001
x9    -3.1500004
x10    2.9999992


## 2. Ma trận chéo trội Cột

### Thuật toán

In [ ]:
def convert_to_iteration_2(A: np.ndarray, 
                           B: np.ndarray, 
                          ) -> Tuple[np.ndarray, np.ndarray]:
    """
    Prepares iteration y_{k+1} = C y_k + D for column-dominant case.
    """

    # 1. T = diag of 1 / diagonal entries of A
    T = np.diag(1.0 / np.diag(A))

    # 2. C = I - A * T
    n = A.shape[0]
    C = np.eye(n) - A.dot(T)
    
    # 3. D = B
    D = B.copy()

    return T, C, D

In [ ]:
def fixed_point_matrix_iteration_2(
    A: np.ndarray,
    T: np.ndarray,
    C: np.ndarray, #main input
    D: np.ndarray,
    x0: Union[np.ndarray, list],
    domiType: int,
    eps: float,
    eta: float
) -> np.ndarray:
    """
    Iterates y_{k+1} = C y_k + D for column-dominant systems.
    
    Parameters:
    - C: (n×n) iteration matrix
    - D: (n,) constant vector
    - y0: initial guess (n,)
    - domiType: dominance type (must be 2 or 3 for column-dominant)
    - eps: relative error tolerance
    
    Uses:
    - vector norm: 1-norm
    - matrix norm: column-sum norm
    - alpha = max|diag(A)| / min|diag(A)| from original A
    
    Prints each iteration's y and error, stops when
      ||y_new - y_old||_1 <= eps * (1 - q) / (alpha * q)
    
    Returns:
    - y_new: approximate solution vector
    """
     # Validate tolerance inputs
    if (eps is None) == (eta is None):
        raise ValueError("Specify exactly one of eps (exact) or eta (relative)")

    # Set norms and α, q based on dominance type
    if domiType in (2,3):
        vec_norm = lambda x: np.sum(np.abs(x))
        q = np.max(np.sum(np.abs(C), axis=0))  # column-sum norm
        diag_vals = np.abs(np.diag(A))         # A should be defined globally
        alpha = np.max(diag_vals) / np.min(diag_vals)
    else:
        raise ValueError("domiType must be 2 for column-dominant iteration")

    # Convergence threshold
    tol = (eps if eps is not None else eta) * (1 - q) / (alpha * q)
    print(f"Shrinking value: {q:.12f}, Additional value: {alpha:.12f}, Threshold: {tol:.12f}")

    # Initialize
    x_old = np.array(x0, dtype=float).flatten()
    T_inv = np.diag(np.diag(A)); y_old = T_inv.dot(x_old)
    history = [np.concatenate([y_old, x_old])]
    errors = [np.nan]

    while True:
        y_new = C.dot(y_old) + D
        x_new = T.dot(y_new)
        if eps is not None:
            err = vec_norm(x_new - x_old)
        else:
            err = vec_norm(x_new - x_old) / vec_norm(x_new)

        # Prepare DataFrame for display
        history.append(np.concatenate([y_new, x_new]))
        errors.append(err)
        y_old = y_new; x_old = x_new;
        if err <= tol:
            break
        
    # Compile DataFrame
    n = len(x_new)
    df = pd.DataFrame(history, columns=[f"y{i+1}" for i in range(n)] + [f"x{i+1}" for i in range(n)])
    df["error"] = errors
    df.index.name = "Iteration"
    return df

### Kết quả

In [ ]:
#Original matrix Ax=B
A = input_matrix('JCB_input_A1.txt', convert_fractions=False)
B = input_matrix('JCB_input_B1.txt', convert_fractions=False).flatten() #remove flatten if B is multi-column matrix

print("\nMatrix A:"); output_matrix(A)
print("\nCheck dominace of A:", check_dominance(A));
print("\nMatrix B:"); output_matrix(B)


Matrix A:
 14.4   0.2   0.6  -0.1   0.9   0.4   0.9
  0.4  16.3  -0.4  -0.7  -0.5   0.1  -0.6
  0.7   0.9  18.6  -0.3   0.6  -0.2   0.4
 -0.3  -1.0  -0.1  19.7  -0.7   0.3  -0.6
  0.6  -0.1  -1.0  -0.6  20.6   0.3  -0.8
  0.4  -0.2  -0.7   0.5  -0.9  22.9   0.2
 -1.0  -0.1   0.5  -0.5   0.2   0.3  23.9

Check dominace of A: 3

Matrix B:
-10.0
 -3.0
 -7.0
  6.0
 -4.0
  1.0
 -7.0


In [ ]:
#Convert to recursion form x_new = Cx+D
T, C, D = convert_to_iteration_2(A, B)

print("\nMatrix C:"); output_matrix(C)
print("\nMatrix D:"); output_matrix(D)


Matrix C:
       0.0 -0.0122699 -0.0322581  0.0050761 -0.0436893 -0.0174672 -0.0376569
-0.0277778        0.0  0.0215054   0.035533  0.0242718 -0.0043668  0.0251046
-0.0486111 -0.0552147        0.0  0.0152284 -0.0291262  0.0087336 -0.0167364
 0.0208333  0.0613497  0.0053763        0.0  0.0339806 -0.0131004  0.0251046
-0.0416667   0.006135  0.0537634  0.0304569        0.0 -0.0131004  0.0334728
-0.0277778  0.0122699  0.0376344 -0.0253807  0.0436893        0.0 -0.0083682
 0.0694444   0.006135 -0.0268817  0.0253807 -0.0097087 -0.0131004        0.0

Matrix D:
-10.0
 -3.0
 -7.0
  6.0
 -4.0
  1.0
 -7.0


In [ ]:
#Calculate the result
domiType = check_dominance(A)
x0 = [0,0,0,0,0,0,0] #initial value
eps = 1e-6
eta = None

df_history = fixed_point_matrix_iteration_2(A, T, C, D, x0, domiType, eps, eta)
print(df_history)


Shrinking value: 0.236111111111, Additional value: 1.659722222222, Threshold: 0.000001949299
                   y1         y2         y3         y4         y5         y6         y7         x1         x2         x3         x4         x5         x6         x7      error
Iteration                                                                                                                                                                      
0           0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000  0.0000000        NaN
1         -10.0000000 -3.0000000 -7.0000000  6.0000000 -4.0000000  1.0000000 -7.0000000 -0.6944444 -0.1840491 -0.3763441  0.3045685 -0.1941748  0.0436681 -0.2928870  2.0901360
2          -9.2860385 -2.9367483 -6.0144809  5.2452282 -4.0427513  0.7090630 -7.3466585 -0.6448638 -0.1801686 -0.3233592  0.2662552 -0.1962501  0.0309634 -0.3073916  0.1740438
3          -9.3024332 -2.97

In [ ]:
solution_series = df_history.filter(regex=r'^x\d+$').iloc[-1]
print("Approximate solution:"),
print(solution_series.to_string())

Approximate solution:
x1   -0.6459132
x2   -0.1822576
x3   -0.3256132
x4    0.2669925
x5   -0.1966741
x6    0.0325279
x7   -0.3070401
